### TODO: 
- branch명은 features/data-preprocessing/[사이트명]
- 배포한 `1차 전처리 정책` 문서를 기반으로 각 사이트에 맞게끔 전처리
- 전처리 이후 스키마 포맷 및 column명은 `정제 데이터 스키마` 문서의 `Table1 스키마 정의`를 따를 것
- DynamoDB적재를 위해  Table1에서 column에 대응되는 key 값이 `-`인 것은 null값이므로 따로 key 생성하지 말 것
- 생성한 json 데이터는 `s3:merged-data-storage`에 적재 (이부분은 기능 구현 후 코드 제공할 예정)

### REQUIRE:
- 최종적인 데이터는 `{"item": {...}}` 형태를 띈 json 포맷이여야 함
- 출처사이트 url, 출처 사이트 심볼 key가 없을 결우 추가할 것(심볼은 Table1 스키마 정의에서 참고)
- 최종적인 key(column) 갯수는 최대 20개로 예상

In [13]:
import json, boto3, datetime, re, pytz
import pandas as pd

In [27]:
# S3 client 생성에 필요한 보안 자격 증명 정보 get
with open("./API_KEYS.json", "r") as f:
    key = json.load(f)

# S3 버킷 정보 get
with open("./DATA_SRC_INFO.json", "r") as f:
    bucket_info = json.load(f)

In [3]:
# S3 섹션 및 client 생성
session = boto3.Session(
    aws_access_key_id=key['aws_access_key_id'],
    aws_secret_access_key=key['aws_secret_key'],
    region_name=key['region']
)

s3 = session.client('s3')

In [28]:
# S3 버킷 정보 init
pull_bucket_name = bucket_info['pull_bucket_name']
push_bucket_name = bucket_info['push_bucket_name']
target_folder_prefix = bucket_info['target_folder_prefix']['wanted_path']

In [15]:
# 특정 폴더 내 파일 목록 가져오기
# TODO: 
# - 마지막 실행일(년,월,일)을 datetime으로 저장한 파일을 읽어들여 curr_date에 적용하기; 당담: 유정연
response = s3.list_objects_v2(Bucket=pull_bucket_name, Prefix=target_folder_prefix, Delimiter='/')
curr_date = datetime.datetime.now(pytz.timezone('Asia/Seoul')).date()  # 로컬 시간대(UTC+9)로 현재 날짜 설정
kst_tz = pytz.timezone('Asia/Seoul') # kst timezone 설정
#curr_date = datetime.date(2024, 8, 21)

# curr_date 보다 날짜가 늦은 data josn 파일 metadata 객체 분류
if 'Contents' in response:
    target_file_list = [obj for obj in response['Contents'] if curr_date <= obj['LastModified'].astimezone(kst_tz).date()]
else:
    print("No objects found in the folder.")

In [22]:
for obj in target_file_list:
    try:
        response = s3.get_object(Bucket=pull_bucket_name, Key=obj['Key'])
        json_context = response['Body'].read().decode('utf-8')
        cleaned_text = re.sub(r'[\r\u2028\u2029]+', ' ', json_context) # 파싱을 위해 unuseal line terminators 제거
        json_list = [json.loads(line) for line in cleaned_text.strip().splitlines()] # pandas format으로 맞추기
        df = pd.DataFrame(json_list)
    except JSONDecodeError as e:
        logging.error(f"JSONDecodeError encountered: {e}")
        continue
    except ClientError as e:
        logging.error(f"ClientError encountered: {e}")
        continue
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        continue
    
    break